<a href="https://colab.research.google.com/github/felixweiland/nft/blob/main/NFT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Goole Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
% cd gdrive/MyDrive/Colab\ Notebooks

In [ ]:
from PIL import Image 
from IPython.display import display 
import random
import json
import pandas as pd
import os
from os import walk
from tqdm import tqdm, tqdm_notebook

In [ ]:
#Defining the user
total_images = int(input("How many would you like to create? "))
user = input("Who is the user? ")
project_name = input("Name of project: ")

#Just making sure that all the correct folders exists
if project_name not in os.listdir(f'/content/gdrive/MyDrive/NFT/{user}/'):
  print("Make sure that you have created a directory with the same name that contains the layers")

for folder in ["images", "meta_data"]:
  if folder not in os.listdir(f'/content/gdrive/MyDrive/NFT/{user}/test/'):
    os.mkdir(f'/content/gdrive/MyDrive/NFT/{user}/test/{folder}')

if "image_data" not in os.listdir(f'/content/gdrive/MyDrive/NFT/{user}/test/meta_data/'):
  os.mkdir(f'/content/gdrive/MyDrive/NFT/{user}/test/meta_data/image_data')

#Defining a path variable to make the code adaptable 
path = f"/content/gdrive/MyDrive/NFT/{user}/{project_name}/"

In [ ]:
#Defining all variables

#First of all I am checking whether or not there is a file already, if this script has been run before in the same project.
#If it has I want to see if the user wants to change any info, or keep the same. If No I import the old data, if Yes we input all again. 
if "project_info.json" in os.listdir(f"/content/gdrive/MyDrive/NFT/{user}/{project_name}/meta_data/"):
  if input("Do you want to change the current info of the project? (Y/N) ") == "Y":
      with open(f"{path}/meta_data/project_info.json") as outfile:
        base_info = json.load(outfile)

        #Printing the current values and asking for new input
        print("Current base_uri: ", base_info["base_uri"])
        base_uri = input("Full url to base-uri: ")

        print(f"Current description: ", base_info["description"])
        description = input("Description of the project: ")

        print(f"Current backgrounds: ", base_info["backgrounds"])
        x = True
        while x:
          truth = []
          backgrounds = input("HEX color codes for background, seperated by commas and without the #: ").split(",")
          for i, item in enumerate(backgrounds):
            backgrounds[i] = item.strip()
            truth.append(len(backgrounds[i])==6)
          if False not in truth:
            x = False
          else:
            print("Please make sure that the input is correct.")

        weights = {}

      #Generating a dictionary for export
      base_info = {"base_uri": base_uri,
                    "project_name": project_name,
                    "description": description,
                    "backgrounds": backgrounds 
      }

      with open(f"{path}/meta_data/project_info.json", "w") as start_file:
        json.dump(base_info, start_file, indent=4)

  else:
    with open(f"{path}/meta_data/project_info.json") as start_file:
      project_data = json.load(start_file)
    
    base_uri = project_data["base_uri"]
    project_name = project_data["project_name"]
    description = project_data["description"]
    backgrounds = project_data["backgrounds"]
    weights = {}

    print("Successfully loaded all data: ")
    for item in project_data.items():
      print(item)

#If there is no project, get the input from the user and then save the file. 
else:
    base_uri = input("Full url to base-uri: ")
    description = input("Description of the project: ")

    #To make sure that the formatting of the HEX is correct
    x = True
    while x:
      truth = []
      backgrounds = input("HEX color codes for background, seperated by commas and without the #: ").split(",")
      for i, item in enumerate(backgrounds):
        backgrounds[i] = item.strip()
        truth.append(len(backgrounds[i])==6)
      if False not in truth:
        x = False

    weights = {}

    base_info = {"base_uri": base_uri,
                  "project_name": project_name,
                  "description": description,
                  "backgrounds": backgrounds 
    }

    with open(f"{path}/meta_data/project_info.json", "w") as start_file:
      json.dump(base_info, start_file, indent=4)

In [ ]:
#Here I want to see if the script has been run before and that there are previously generated images. I don't want any doubles, so if there is
#I will import that file as all_images. If not all_images is an empty list. 
try: 
  with open(f"{path}/meta_data/all-traits.json") as outfile:
    all_images = json.load(outfile)
    #images_to_generate is a list that will contain all new pictures that are to be generated. 
    images_to_generate = []
    #Here I am taking in a number to be used later in order to filter out already inputed items.
    delete_past = len(all_images)

except:
  all_images = []
  images_to_generate = []
  delete_past = 0

In [ ]:
dirpaths = []
layers = []
clean_layers = []
path_dictionary = {}

#A function that creates all the baseline stuff. I get all different folders available and all the files that they contain.
#I append them to the above stated lists and dictionaries. 
def gen_paths(path):
    
    """
    Generates all the folders nessecary for the program. Just insert the folder path to the layers as a string.
    
    """
    
    #Creating a list with all the paths and making sure that they are in the correct order. 
    for (dirpath, dirnames, filenames) in walk(f'{path}layers/'):
        if dirpath not in dirpaths:
            dirpaths.append(dirpath)
    dirpaths.sort()
        
    #Appending all filenames to each respective folder in the path
    for (dirpath, dirnames, filenames) in walk(f'{path}layers/'):

        dirpath = dirpath.split("/")

        if len(dirpath[-1]) > 2:
          path_dictionary[dirpath[-1]] = []
            
    #Creating a list with all the layers
    for item in path_dictionary.keys():
        if item not in layers:
            layers.append(item)
    layers.sort()

    #And one that is without the order
    for item in layers:
        if item not in clean_layers:
            clean_layers.append(item.split(". ")[1])
    
    #Appending all file names to the respective folder
    for item in path_dictionary.keys():
    
        for (dirpath, dirnames, filenames) in walk(f"{path}/layers/{item}"):
            path_dictionary[item].extend(filenames)
            break
            
gen_paths(path)

In [ ]:
# Creating a dictionary for different weights.
def gen_weights():
    
    """
    Used to calculate different weights for attributes. Iterates over each layer and takes user input. Input - to create equal weights.
    """
    print("Input weights for the attributes and seperate by comma.\nThey need to add up to 100 and have the same number of items as printed.\nInput - if you want to have the same probabiltiy for all")
    
    #Iterating over each key in the path_dictionary.
    for key in path_dictionary.keys():

        #A placeholder list
        item_list = []
        error = True

        #Taking input from the user regarding the weights
        while error:
          w_input = input(f"{len(path_dictionary[key])} items. \nWeights for {path_dictionary[key]}: ")

          if len(w_input.split(",")) == len(path_dictionary[key]) or w_input == "-":
            error = False
          else:
            print("Not valid input.. Try again.")

        #Checking to see if the user wants to have the same weights.
        if w_input != "-":
            for item in w_input.split(","):
                item_list.append(int(item))

        #Calculating the weights and appending them to the list
        else:
            item_list.append(100/len(path_dictionary[key]))
            item_list = item_list*len(path_dictionary[key])

        #Assingning the list to each respective key.
        weights[key] = item_list
        
    #Exporting the file
    data_file = f'{path}/meta_data/weights.json';
    with open(data_file, 'w') as outfile:
      json.dump(weights, outfile, indent=4)

In [ ]:
try:
  with open(f"{path}/meta_data/weights.json") as outfile:
    weights = json.load(outfile)
    for item in weights.items():
      print(item)

  if input("Do you want to change the weights from the previous settings? (Y/N)") == "Y":
    gen_weights()
  
  else:
    print("Proceeding with the current weights")

except:
  gen_weights()

In [ ]:
def create_new_image():
    
    #Creating a placeholder dictionary
    new_image = {} 
    new_image[f"{1 + len(path_dictionary.keys())}. tokenId"] = delete_past+i
    new_image[f"{2 + len(path_dictionary.keys())}. background"] = random.choices(backgrounds)[0]

    for key in path_dictionary.keys():

        #Creating keys without the positioning numbers
        new_image[key] = random.choices(path_dictionary[key], weights[key])[0]
        
    #Return the dictionary if not already in all_images. This to prevet doubles. 
    if new_image not in all_images:
        return new_image 
    
    #If the image already exists - run the function again.
    else:
        return create_new_image()

# Generate the unique combinations based on trait weightings and appending an unique ID
for i in range(total_images):

    new_trait_image = create_new_image()
    all_images.append(new_trait_image)
    images_to_generate.append(new_trait_image)

    #Generating a random background.
    background = random.choices(backgrounds)[0]

In [ ]:
#Just to make sure that the images that are to be generated are the same as in the all_images file. 
def sanity_check():
  for i in range(len(images_to_generate)):
    truth = []
    truth.append(images_to_generate[i] == all_images[delete_past+i])
  if False not in truth:
    print("All checks out")

  else:
    raise SanityError("There seems to be some differences, please look over the content again")

sanity_check()

In [ ]:
#Cleaning up the attributes for export
meta_data = []

for item in all_images:
    
    data = {}
    
    for key, value in item.items():

      #To remove the file-type from the data files
      if str(value)[-4:] == ".png" or str(value)[-4:] == ".jpg":
          try:
            data[key.split(". ")[1]] = value.split(".")[0]
          except:
            data[key] = value.split(".")[0]

      #For all values that does not have a file appendix
      else:
          try:
            data[key.split(". ")[1]] = value
          except:
            data[key] = value
        
    meta_data.append(data)

#Creating a json-file with the traits.
meta_data_file = f'{path}/meta_data/all-traits.json'; 
with open(meta_data_file, 'w') as outfile:
  json.dump(meta_data, outfile, indent=4)

#Creating an excel sheet with all information - for the user to get an overview.
excel_export = pd.DataFrame(meta_data)
excel_export.to_excel(f'{path}/meta_data/all-traits.xlsx')

In [ ]:
def gen_image():
    
    #Iterating over all items in all_images dictionary
    for item in tqdm(images_to_generate):
        
        #Creating an empty dictonary to hold all the image-files for each layer of the composition. Resets for every new one.
        image = {}
        
        #Iterating over the keys together with an index locator appending them to the path-dictionary with the index as a key
        #This will help in the later execution of the code.
        try:
          for i, key in enumerate(path_dictionary.keys()):  
              image[i] = Image.open(f'{dirpaths[i+1]}/{item[layers[i]]}').convert('RGBA')

        except:
          for i, key in enumerate(layers):  
              image[i] = Image.open(f'{dirpaths[i+1]}/{item[clean_layers[i]]}').convert('RGBA')

        #Creating a baseline image with the first and second element of the picture
        composite_image = Image.alpha_composite(image[0], image[1])

        #Iterating over the keys and values in the image-dictionary to extract the specific image-layer
        for key, value in image.items():
            
            #Since we used the first two index counts, we restrict this loop while the key is less than the
            #lenght of image. This helps the script from breaking while allowing any number of layers.
            if key < len(image)-2:
                
                #Continously adding the different layers to the composition
                composite_image = Image.alpha_composite(composite_image, image[key+2])
                
        #Saving the composition as the tokenId in png-format
        final_image = composite_image.convert('RGB')
        try:
          file_name = str(item[f'{1+len(path_dictionary.keys())}. tokenId']) + ".png"
        except:  
          file_name = str(item['tokenId']) + ".png"
        final_image.save(f"{path}/images/" + file_name)
        
gen_image()

In [ ]:
#Function that takes in a key and a value and returns them inside a dictionary with keys "trait-type" and "value"
def getAttribute(key, value):
    """
    Function that takes in a key and a value and returns them inside a dictionary with keys "trait-type" and "value"
    """
    return {"trait_type": key, "value": value}

#Function that generates the meta-data per image and saves it as a json-file
def yield_attributes(data_file):
    """
    Yields NFT-attributes from json-file. Enter path to file.
    """
    file = open(data_file,) 
    data = json.load(file)

    #Iterating over the dictonaries in the data file while excluding previously generated files.
    for idx, item in enumerate(data[delete_past:]):
        token_id = item['tokenId']

        #Because there are two ways that the background is named I added this error handling. 
        try:
          token = {
              "image": base_uri + str(token_id) + '.png',
              "tokenId": token_id,
              "name": project_name + ' #' + str(token_id),
              "description":description,
              "background_color":all_images[idx][f"{2 + len(path_dictionary.keys())}. background"],
              "attributes": []
          }

        except:
          token = {
            "image": base_uri + str(token_id) + '.png',
            "tokenId": token_id,
            "name": project_name + ' #' + str(token_id),
            "description":description,
            "background_color":all_images[idx]["background"],
            "attributes": []
          }

        for i, _ in enumerate(clean_layers):
            token["attributes"].append(getAttribute(clean_layers[i], item[clean_layers[i]]))

        with open(f'{path}/meta_data/image_data/' + str(token_id) +".json", 'w') as outfile:
            json.dump(token, outfile, indent=4)
    file.close()
    
yield_attributes(f'{path}/meta_data/all-traits.json')